You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [28]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [29]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [30]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [31]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [32]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [33]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [34]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [35]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [36]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [37]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-01-20 15:45:29,663 - 28516 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [38]:
result = crew.kickoff(inputs={"topic": "AI Agents for Sales Automation in Interior Design & Home Decor"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Agents for Sales Automation in Interior Design & Home Decor.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: AI Agents for Sales Automation in Interior Design & Home Decor

Outline:
I. Introduction
- Definition of AI agents for sales automation in interior design & home decor
- Importance of AI agents in the industry
- Brief overview of key players in the market

II. Latest Trends in AI Agents for Sales Automation
- Personalization and customization features
- Integration with virtual reality technology
- Data-driven decision making
- Enhanced customer engagement 

- Display the results of your execution as markdown in the notebook.

In [39]:
from IPython.display import Markdown
Markdown(result)

# AI Agents for Sales Automation in Interior Design & Home Decor

In today's rapidly evolving digital landscape, AI agents are revolutionizing the way sales automation is approached in the interior design and home decor industry. These intelligent agents utilize advanced algorithms and data analytics to streamline processes, enhance customer engagement, and drive sales efficiency. From personalized recommendations to virtual reality integration, AI agents are reshaping the way businesses interact with their customers and optimize their sales strategies.

## Latest Trends in AI Agents for Sales Automation

One of the key trends in AI agents for sales automation in interior design and home decor is the focus on personalization and customization features. By leveraging customer data and preferences, AI agents can recommend tailored design solutions and products, creating a more engaging and personalized shopping experience for consumers. Additionally, the integration of virtual reality technology allows customers to visualize products in their own spaces, increasing confidence in purchasing decisions and reducing returns.

Data-driven decision making is another crucial trend in AI sales automation, as businesses can analyze customer behavior and trends to optimize their marketing strategies and product offerings. Furthermore, AI chatbots are enhancing customer engagement by providing instant support and assistance, improving overall satisfaction and driving repeat business.

## Key Players in the Market

Leading companies in the market offering AI solutions for interior design and home decor sales include industry giants like Houzz and Wayfair. These companies have successfully implemented AI agents to streamline their sales processes, improve customer experience, and drive revenue growth. Case studies showcasing the impact of AI agents on the industry highlight the significant improvements in sales efficiency and customer satisfaction, demonstrating the tangible benefits of adopting AI technology in the interior design and home decor sector.

## Noteworthy News in the Industry

Recent developments in AI technology for sales automation have shown promising results in improving sales efficiency and customer satisfaction. By leveraging AI agents, businesses can gain valuable insights into customer preferences, optimize their product offerings, and enhance the overall shopping experience. The future prospects and trends to watch out for in the market include further advancements in AI chatbot technology, increased integration with virtual reality, and a continued focus on data-driven decision making.

## Target Audience Analysis

The target audience for AI agents in interior design and home decor sales includes homeowners seeking personalized design solutions, interior designers looking for efficient sales automation tools, and home decor retailers aiming to enhance customer experience and boost sales. By understanding the unique needs and pain points of these target groups, businesses can tailor their AI solutions to address specific challenges and drive growth in the competitive industry.

In conclusion, AI agents for sales automation in interior design and home decor are revolutionizing the way businesses interact with customers, optimize sales processes, and drive revenue growth. By staying abreast of the latest trends, key players in the market, and noteworthy news in the industry, businesses can harness the power of AI technology to stay competitive and meet the evolving needs of their target audience.

*Call to Action:*
As the industry continues to evolve, it is essential for businesses in the interior design and home decor sector to explore AI solutions for their sales automation needs. For further information and consultation, consider reaching out to leading AI providers in the market to see how AI agents can transform your business operations and drive success in the digital age.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [40]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: Exploring the Latest Trends in Sustainable Fashion

Outline:
I. Introduction
- Brief overview of sustainable fashion
- Importance of being mindful of environmental impact
II. Latest Trends
- Rise of eco-friendly materials like organic cotton and recycled fabrics
- Growing popularity of slow fashion and minimalist wardrobes
- Innovation in sustainable production techniques such as zero-waste manufacturing
III. Key Players
- Sustainable fashion brands leading the industry
- Influencers promoting 

In [41]:
Markdown(result)

# Exploring the Latest Trends in Sustainable Fashion

In recent years, there has been a significant shift towards sustainable fashion as consumers become more aware of the environmental impact of the clothing industry. The concept of sustainable fashion revolves around creating clothing in an ethical and environmentally friendly manner, from the sourcing of materials to the production process. It is essential for individuals to be mindful of their purchasing decisions and opt for brands that prioritize sustainability.

## Latest Trends

One of the latest trends in sustainable fashion is the rise of eco-friendly materials such as organic cotton and recycled fabrics. These materials not only reduce the carbon footprint of clothing production but also promote a healthier environment. Additionally, there has been a growing popularity of slow fashion and minimalist wardrobes, encouraging consumers to invest in high-quality pieces that are designed to last.

Innovation in sustainable production techniques is also on the rise, with the introduction of zero-waste manufacturing processes. These techniques aim to minimize waste during the production process, ultimately reducing the environmental impact of clothing production. By embracing these trends, the fashion industry can move towards a more sustainable future.

## Key Players

Sustainable fashion brands play a crucial role in leading the industry towards more ethical and eco-friendly practices. These brands prioritize transparency in their supply chain and are committed to reducing their environmental impact. Influencers also play a significant role in promoting ethical and sustainable fashion choices to their followers, encouraging them to make conscious purchasing decisions.

Furthermore, organizations advocating for sustainable practices in the fashion industry are instrumental in driving change. They work towards raising awareness about the environmental and social impact of the clothing industry, pushing for more sustainable practices from brands and consumers alike.

## Noteworthy News

Recent collaborations between designers and sustainable brands have brought attention to the importance of sustainable fashion initiatives. These collaborations showcase the possibilities of creating stylish and innovative clothing while prioritizing sustainability. The impact of sustainable fashion initiatives on reducing the carbon footprint of the industry is evident, as more brands and consumers embrace eco-friendly practices.

As consumers become more conscious of the environmental impact of the fashion industry, it is crucial for them to support sustainable fashion brands and make informed purchasing decisions. By incorporating sustainable practices into their wardrobe, individuals can contribute to a more sustainable future for the fashion industry and the planet as a whole.

## Target Audience

Millennials and Gen Z consumers are at the forefront of the sustainable fashion movement, showing a keen interest in ethical and eco-friendly products. Fashion enthusiasts looking to make conscious purchasing decisions are also part of the target audience for sustainable fashion content. Individuals who are concerned about the environmental impact of the fashion industry can benefit from learning more about sustainable practices and supporting brands that prioritize sustainability.

In conclusion, exploring the latest trends in sustainable fashion is essential for individuals looking to make a positive impact on the environment through their clothing choices. By supporting sustainable fashion brands, incorporating eco-friendly materials into their wardrobe, and advocating for more sustainable practices in the industry, consumers can contribute to a more ethical and environmentally friendly fashion landscape. Let's work together towards a more sustainable future for the fashion industry and our planet.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).